In [1]:
import soundfile as sf
import os, librosa
import numpy as np
import torch
import torch.nn as nn
import random
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import timeit
from pynvml import *
# import opensmile
from scipy import signal
import pwcca
import torch.optim as optim

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
asr_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(DEVICE)

/afs/inf.ed.ac.uk/user/s20/s2057508/.local/lib/python3.6/site-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.codevectors', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

In [3]:
# smile = opensmile.Smile(
#     feature_set=opensmile.FeatureSet.eGeMAPSv02,
#     feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
# )

# smile2 = opensmile.Smile(
#     feature_set=opensmile.FeatureSet.eGeMAPSv02,
#     feature_level=opensmile.FeatureLevel.Functionals,
# )

smile3 = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

smile4 = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors_Deltas,
)

# smile5 = opensmile.Smile(
#     feature_set=opensmile.FeatureSet.ComParE_2016,
#     feature_level=opensmile.FeatureLevel.Functionals,
# )

In [43]:
# batch data preparation CNN+lstm, frame-level
path = "/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/IEMOCAP/Data"
os.chdir(path)
names_train = []
names_test = []
emolabels_train = []
emolabels_test = []

# train_hct_feats = []
# test_hct_feats = []
# train_mfcc_feats = []
# test_mfcc_feats = []

train_asr_hidden0 = []
train_asr_hidden1 = []
train_asr_hidden2 = []
train_asr_hidden3 = []
train_asr_hidden4 = []
train_asr_hidden5 = []
train_asr_hidden6 = []
train_asr_hidden7 = []
train_asr_hidden8 = []
train_asr_hidden9 = []
train_asr_hidden10 = []
train_asr_hidden11 = []
train_asr_hidden12 = []
train_asr_hiddenave = []
train_asr_hiddenrm = []

test_asr_hidden0 = []
test_asr_hidden1 = []
test_asr_hidden2 = []
test_asr_hidden3 = []
test_asr_hidden4 = []
test_asr_hidden5 = []
test_asr_hidden6 = []
test_asr_hidden7 = []
test_asr_hidden8 = []
test_asr_hidden9 = []
test_asr_hidden10 = []
test_asr_hidden11 = []
test_asr_hidden12 = []
test_asr_hiddenave = []
test_asr_hiddenrm = []

with open('emo_name_trans_5500.txt', 'r') as f:
    files = f.readlines()
    files = np.array(files)
    indices = np.arange(len(files))
    random.shuffle(indices)
    files[np.arange(len(files))] = files[indices]
    n = 1
    for line in files:
        print(n)
        emotion = line.split(" ")[0]
        file_name = line.split(" ")[1]
        
        if emotion == "Ang":
            emotion_label = 0
        elif emotion == "Hap":
            emotion_label = 1
        elif emotion == "Neu":
            emotion_label = 2
        elif emotion == "Sad":
            emotion_label = 3
        
        torch.cuda.empty_cache()
        
#         audio_input, sr = sf.read(file_name)
        audio_input, sr = librosa.load(file_name, sr=16000)
#         audio_input_hct = librosa.util.fix_length(audio_input, size=120000)
        
        if len(audio_input) > 400000:
            continue
        inputs = processor(audio_input, sampling_rate=16000, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)

        hidden = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10 + hidden11 + hidden12) / 13.0
        hidden_rm = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10) / 11.0

    #     hct features
#         mfcc = librosa.feature.melspectrogram(y=audio_input, sr=sr, n_fft=400, hop_length=160).T
#         mfcc = np.mean(mfcc, axis=0)
#         feats1 = smile.process_signal(audio_input, sr).values
#         feats2 = smile2.process_signal(audio_input, sr).values
#         feats3 = smile3.process_signal(audio_input, sr).values
#         feats4 = smile4.process_signal(audio_input, sr).values
#         feats5 = smile5.process_signal(audio_input, sr).values.reshape(1, -1)

#         feats1 = np.mean(feats1, axis=0)
#         feats2 = np.mean(feats2, axis=0)
#         feats3 = np.mean(feats3, axis=0)
#         feats4 = np.mean(feats4, axis=0)
#         mfcc = mfcc.reshape(1, -1)
#         hct = np.concatenate((feats3, feats4)).reshape(1, -1)
#         hidden = hidden.reshape(1, -1)

#         print(hct.shape, hidden.shape)
             
        if "Ses05" not in file_name:
            names_train.append(file_name)
            emolabels_train.append(emotion_label)
            train_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            train_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            train_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            train_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            train_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            train_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            train_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            train_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            train_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            train_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            train_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            train_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            train_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())
            train_asr_hiddenave.append(hidden.view(1, 768).cpu().numpy())
            train_asr_hiddenrm.append(hidden_rm.view(1, 768).cpu().numpy())
#             train_hct_feats.append(np.array(hct))
#             train_mfcc_feats.append(np.array(mfcc))
                
        elif "Ses05" in file_name:
            names_test.append(file_name)
            emolabels_test.append(emotion_label)
            test_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            test_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            test_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            test_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            test_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            test_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            test_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            test_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            test_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            test_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            test_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            test_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            test_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())
            test_asr_hiddenave.append(hidden.view(1, 768).cpu().numpy())
            test_asr_hiddenrm.append(hidden_rm.view(1, 768).cpu().numpy())
#             test_hct_feats.append(np.array(hct))
#             test_mfcc_feats.append(np.array(mfcc))

        n+=1            
        torch.cuda.empty_cache()
f.close()

emolabels_train = np.array(emolabels_train)
emolabels_test = np.array(emolabels_test)
# train_hct_feats = torch.tensor(train_hct_feats).to(DEVICE)
# test_hct_feats = torch.tensor(test_hct_feats).to(DEVICE)
# train_mfcc_feats = torch.tensor(train_mfcc_feats).to(DEVICE)
# test_mfcc_feats = torch.tensor(test_mfcc_feats).to(DEVICE)
train_asr_hidden0 = torch.tensor(train_asr_hidden0).to(DEVICE)
train_asr_hidden1 = torch.tensor(train_asr_hidden1).to(DEVICE)
train_asr_hidden2 = torch.tensor(train_asr_hidden2).to(DEVICE)
train_asr_hidden3 = torch.tensor(train_asr_hidden3).to(DEVICE)
train_asr_hidden4 = torch.tensor(train_asr_hidden4).to(DEVICE)
train_asr_hidden5 = torch.tensor(train_asr_hidden5).to(DEVICE)
train_asr_hidden6 = torch.tensor(train_asr_hidden6).to(DEVICE)
train_asr_hidden7 = torch.tensor(train_asr_hidden7).to(DEVICE)
train_asr_hidden8 = torch.tensor(train_asr_hidden8).to(DEVICE)
train_asr_hidden9 = torch.tensor(train_asr_hidden9).to(DEVICE)
train_asr_hidden10 = torch.tensor(train_asr_hidden10).to(DEVICE)
train_asr_hidden11 = torch.tensor(train_asr_hidden11).to(DEVICE)
train_asr_hidden12 = torch.tensor(train_asr_hidden12).to(DEVICE)
train_asr_hiddenave = torch.tensor(train_asr_hiddenave).to(DEVICE)
train_asr_hiddenrm = torch.tensor(train_asr_hiddenrm).to(DEVICE)

test_asr_hidden0 = torch.tensor(test_asr_hidden0).to(DEVICE)
test_asr_hidden1 = torch.tensor(test_asr_hidden1).to(DEVICE)
test_asr_hidden2 = torch.tensor(test_asr_hidden2).to(DEVICE)
test_asr_hidden3 = torch.tensor(test_asr_hidden3).to(DEVICE)
test_asr_hidden4 = torch.tensor(test_asr_hidden4).to(DEVICE)
test_asr_hidden5 = torch.tensor(test_asr_hidden5).to(DEVICE)
test_asr_hidden6 = torch.tensor(test_asr_hidden6).to(DEVICE)
test_asr_hidden7 = torch.tensor(test_asr_hidden7).to(DEVICE)
test_asr_hidden8 = torch.tensor(test_asr_hidden8).to(DEVICE)
test_asr_hidden9 = torch.tensor(test_asr_hidden9).to(DEVICE)
test_asr_hidden10 = torch.tensor(test_asr_hidden10).to(DEVICE)
test_asr_hidden11 = torch.tensor(test_asr_hidden11).to(DEVICE)
test_asr_hidden12 = torch.tensor(test_asr_hidden12).to(DEVICE)
test_asr_hiddenave = torch.tensor(test_asr_hiddenave).to(DEVICE)
test_asr_hiddenrm = torch.tensor(test_asr_hiddenrm).to(DEVICE)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346


In [28]:
print(train_hct_feats.shape)
print(test_hct_feats.shape)
print(train_mfcc_feats.shape)
print(test_mfcc_feats.shape)

(4270, 1, 258)
(1225, 1, 258)
(4270, 1, 128)
(1225, 1, 128)


In [17]:
train_hct_feats = np.concatenate((train_hct_feats, train_mfcc_feats), axis=2)
test_hct_feats = np.concatenate((test_hct_feats, test_mfcc_feats), axis=2)

In [27]:
train_hct_feats = train_hct_feats.cpu().numpy()
test_hct_feats = test_hct_feats.cpu().numpy()

In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=768)
pca.fit(X_new_train)
X_new_train = pca.transform(X_new_train)
X_new_test = pca.transform(X_new_test)

ValueError: Found array with dim 3. Estimator expected <= 2.

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC


train_hct_feats = train_hct_feats.cpu().numpy()
test_hct_feats = test_hct_feats.cpu().numpy()
scaler = StandardScaler()
scaler.fit(train_hct_feats)
sel_ = SelectFromModel(LinearSVC(C=1, penalty='l2'))
scaler.transform(train_hct_feats, emolabels_train)

sel_.fit(train_hct_feats, emolabels_train)

X_train_selected = sel_.transform(train_hct_feats)
X_test_selected = sel_.transform(test_hct_feats)

X_train_selected = torch.from_numpy(X_train_selected)
X_test_selected = torch.from_numpy(X_test_selected)

X_train_selected = X_train_selected.view(4270, 1, 64)
X_test_selected = X_test_selected.view(1225, 1, 64)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [29]:
train_hct_feats -= np.mean(train_hct_feats, axis = 0)
train_hct_feats /= np.std(train_hct_feats, axis = 0)
test_hct_feats -= np.mean(test_hct_feats, axis = 0)
test_hct_feats /= np.std(test_hct_feats, axis = 0)

In [31]:
train_hct_feats = torch.from_numpy(train_hct_feats)
test_hct_feats = torch.from_numpy(test_hct_feats)

In [6]:
indices_train = []
for i in range(len(names_train)):
    indices_train.append(i)

In [5]:
print(len(names_test)+len(names_train))

5495


In [45]:
#MLP
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
#         self.amp = nn.Linear(90, 768)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.dense1 = nn.Linear(768, 128)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 4)

    def forward(self, x):
#         x = self.amp(x)
#         self.lstm.flatten_parameters()
#         x, _ = self.lstm(x)
#         x, _ = self.attn(x,x,x)
        x = self.flat(x)
        x = self.dense1(x)
        x = self.acti(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion

In [41]:
# weighted sum
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.W = torch.nn.Parameter(torch.ones(1, 13), requires_grad=True)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.dense1 = nn.Linear(768, 128)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 4)

    def forward(self, x):
        x = torch.matmul(self.W, x)
        x = self.flat(x)
        x = torch.div(x, torch.sum(self.W, 1))
#         print(x.size(), self.W.size())
#         print(torch.sum(self.W, 2).size())
#         x = self.amp(x)
#         self.lstm.flatten_parameters()
#         x, _ = self.lstm(x)
#         x, _ = self.attn(x,x,x)
#         x = self.flat(x)
        x = self.dense1(x)
        x = self.acti(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion, self.W

In [78]:
# weighted sum rm 2
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.W = torch.nn.Parameter(torch.ones(1, 11), requires_grad=True)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.dense1 = nn.Linear(768, 128)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 4)

    def forward(self, x):
        x = torch.matmul(self.W, x)
        x = self.flat(x)
        x = torch.div(x, torch.sum(self.W, 1))
#         print(x.size(), self.W.size())
#         print(torch.sum(self.W, 2).size())
#         x = self.amp(x)
#         self.lstm.flatten_parameters()
#         x, _ = self.lstm(x)
#         x, _ = self.attn(x,x,x)
#         x = self.flat(x)
        x = self.dense1(x)
        x = self.acti(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion, self.W

In [90]:
#concatenation
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
#         self.amp = nn.Linear(90, 768)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)        
        self.lstm2 = nn.LSTM(input_size=90,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.norm = nn.LayerNorm(128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out= nn.Linear(16, 4)

    def forward(self, x, y):
#         y = self.amp(y)
        self.lstm.flatten_parameters()
        self.lstm2.flatten_parameters()        
        x, _ = self.lstm(x)
        y, _ = self.lstm2(y)
        cat = torch.cat((x, y), 2)
        cat = self.norm(cat)
        z, _ = self.attn(cat,cat,cat)
        z = self.flat(z)
        z = self.dense(z)
        res = self.acti(z)
        emotion = self.out(res)
        return emotion

In [101]:
#co-attention
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=90,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.amp = nn.Linear(64, 128)
        self.norm = nn.LayerNorm(256)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(256, 16)
        self.acti = nn.ReLU()
        self.out= nn.Linear(16, 4)

    def forward(self, x, y):
        self.lstm.flatten_parameters()
        self.lstm2.flatten_parameters()
        x, _ = self.lstm(x)
        y, _ = self.lstm2(y)
        x = self.amp(x)
        y = self.amp(y)
        z1, _ = self.attn(x,y,y)
        z2, _ = self.attn(y,x,x)
        cat = torch.cat((z1, z2), 2)
        cat = self.norm(cat)
        cat = self.flat(cat)
        cat = self.dense(cat)
        res = self.acti(cat)
        emotion = self.out(res)
        return emotion

In [103]:
#cross-domain fusion
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=90,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(512, 16)
        self.acti = nn.ReLU()
        self.out= nn.Linear(16, 4)
        
    def forward(self, x, y):
        # lstm
        self.lstm.flatten_parameters()
        self.lstm2.flatten_parameters()
        x, _ = self.lstm(x)
        y, _ = self.lstm2(y)
        
        x = self.amp(x)
        y = self.amp(y)
        
        # attention
        x_self, _ = self.attn(x,x,x)
        y_self, _ = self.attn(y,y,y)
        x_y, _ = self.attn(x,y,y)
        y_x, _ = self.attn(y,x,x)

        # distillation loss
        loss_dis1 = MSE_func(x_y, y_x)
        loss_dis2 = MSE_func(y_x, x_y)

        # similarity loss
        loss_sim1 = kl_func(nn.functional.log_softmax(x_y, 0), nn.functional.softmax(x_self, 0))
        loss_sim2 = kl_func(nn.functional.log_softmax(y_x, 0), nn.functional.softmax(y_self, 0))

        # concatenation
        x_co = torch.cat((x_self, y_self, x_y, y_x), 2)
        x_co = self.flat(x_co)
#         x_co = x_co.mean(dim=1)  # pooling
        x_co = self.dense(x_co)
        x_co = self.acti(x_co)
        emotion = self.out(x_co)
        return emotion

In [106]:
#hierarchical cross-attention

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=90,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(768, 16)
        self.acti = nn.ReLU()
        self.out= nn.Linear(16, 4)
        
    def forward(self, x1, x2, x3, y):
        self.lstm.flatten_parameters()
        self.lstm2.flatten_parameters()
        x1, _ = self.lstm(x1)
        x2, _ = self.lstm(x2)
        x3, _ = self.lstm(x3)
        y, _ = self.lstm2(y)
        
        x1 = self.amp(x1)
        x2 = self.amp(x2)
        x3 = self.amp(x3)
        y = self.amp(y)
        
        z1, _ = self.attn(x1,y,y)
        z2, _ = self.attn(y,x1,x1)

        z11, _ = self.attn(x2,y,y)
        z22, _ = self.attn(y,x2,x2)
        
        z111, _ = self.attn(x3,y,y)
        z222, _ = self.attn(y,x3,x3)
        
        cat = torch.cat((z1, z2, z11, z22, z111, z222), 2)
        cat = self.flat(cat)
        
#         cat = cat.mean(dim=1)
        cat = self.dense(cat)
        res = self.acti(cat)
        emotion = self.out(res)
        return emotion

In [110]:
#late fusion

class NeuralNet1(nn.Module):
    def __init__(self):
        super(NeuralNet1,self).__init__()
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(64, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(64, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 4)

    def forward(self, x):
        self.lstm.flatten_parameters()
        x, _ = self.lstm(x)
#         x = self.amp(x)
        x, _ = self.attn(x,x,x)
        x = self.flat(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion

In [111]:
#late fusion
class NeuralNet2(nn.Module):
    def __init__(self):
        super(NeuralNet2,self).__init__()
        self.lstm = nn.LSTM(input_size=90,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(64, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(64, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 4)

    def forward(self, x):
        self.lstm.flatten_parameters()
        x, _ = self.lstm(x)
#         x = self.amp(x)
        x, _ = self.attn(x,x,x)
        x = self.flat(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion

In [ ]:
torch.manual_seed(1)
ser_model = NeuralNet().to(DEVICE)
params = ser_model.parameters()
optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
# MSE_func = nn.MSELoss()
# kl_func = nn.KLDivLoss(reduction='batchmean')

func = nn.CrossEntropyLoss()

epochs = 500
epoch = 0
batch_size = 64


# training
while epoch < epochs:
    torch.cuda.empty_cache()
#     start = timeit.default_timer()
#     print("-----epoch: ", epoch, "-----")
    predictions_train = []
    predictions_test = []

#     print('--training begins--')
    ser_model.train()
    j = 0
    while j < len(names_train)/batch_size:
        if (j+1)*batch_size > len(names_train):
            emolabels = torch.tensor(emolabels_train[j*batch_size: ]).to(DEVICE)
            indices = indices_train[j*batch_size: ]
            asr_feats0 = train_asr_hidden0[j*batch_size: ].to(DEVICE)
            asr_feats1 = train_asr_hidden1[j*batch_size: ].to(DEVICE)
            asr_feats2 = train_asr_hidden2[j*batch_size: ].to(DEVICE)
            asr_feats3 = train_asr_hidden3[j*batch_size: ].to(DEVICE)
            asr_feats4 = train_asr_hidden4[j*batch_size: ].to(DEVICE)
            asr_feats5 = train_asr_hidden5[j*batch_size: ].to(DEVICE)
            asr_feats6 = train_asr_hidden6[j*batch_size: ].to(DEVICE)
            asr_feats7 = train_asr_hidden7[j*batch_size: ].to(DEVICE)
            asr_feats8 = train_asr_hidden8[j*batch_size: ].to(DEVICE)
            asr_feats9 = train_asr_hidden9[j*batch_size: ].to(DEVICE)
            asr_feats10 = train_asr_hidden10[j*batch_size: ].to(DEVICE)
            asr_feats11 = train_asr_hidden11[j*batch_size: ].to(DEVICE)
            asr_feats12 = train_asr_hidden12[j*batch_size: ].to(DEVICE)
            asr_featsave = train_asr_hiddenave[j*batch_size: ].to(DEVICE)
            asr_featsrm = train_asr_hiddenrm[j*batch_size: ].to(DEVICE)
            
        else:
            emolabels = torch.tensor(emolabels_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            indices = indices_train[j*batch_size:(j+1)*batch_size]
            asr_feats0 = train_asr_hidden0[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats1 = train_asr_hidden1[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats2 = train_asr_hidden2[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats3 = train_asr_hidden3[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats4 = train_asr_hidden4[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats5 = train_asr_hidden5[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats6 = train_asr_hidden6[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats7 = train_asr_hidden7[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats8 = train_asr_hidden8[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats9 = train_asr_hidden9[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats10 = train_asr_hidden10[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats11 = train_asr_hidden11[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats12 = train_asr_hidden12[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_featsave = train_asr_hiddenave[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_featsrm = train_asr_hiddenrm[j*batch_size:(j+1)*batch_size].to(DEVICE)
            
        asr_weighted = torch.cat((asr_feats0,asr_feats1,asr_feats2,asr_feats3,asr_feats4,asr_feats5,asr_feats6,
                                 asr_feats7,asr_feats8,asr_feats9,asr_feats10), 1)
            
# ser loss
        pred, weights = ser_model(asr_weighted)
#         pred = ser_model(asr_featsrm)
#         pred = ser_model(hct_feats)
#         pred = ser_model(asr_feats0, hct_feats)
#         pred = ser_model(asr_feats_g1, asr_feats_g2, asr_feats_g3, hct_feats.to(torch.float))
        train_ser_loss = func(pred, torch.tensor(emolabels))
        for i in pred:
            predictions_train.append(i.detach().cpu().numpy())
        j += 1

# backprop
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(params, 1.0)
        train_ser_loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
#     print('--training ends--')


# testing
# extract features
#     print('--testing begins--')
    ser_model.eval()

# ser loss
    test_asr_weighted = torch.cat((test_asr_hidden0,test_asr_hidden1,test_asr_hidden2,test_asr_hidden3,test_asr_hidden4,
                                 test_asr_hidden5,test_asr_hidden6,test_asr_hidden7,test_asr_hidden8,test_asr_hidden9,
                                 test_asr_hidden10), 1)
    pred, weights = ser_model(test_asr_weighted)
#     pred = ser_model(test_asr_hiddenrm)
#     pred = ser_model(test_hct_feats.to(DEVICE))
#     pred = ser_model(test_asr_hidden0, test_hct_feats.to(DEVICE))
#     asr_feats_g1 = (test_asr_hidden1 + test_asr_hidden2 + test_asr_hidden3 + test_asr_hidden4) / 4.0
#     asr_feats_g2 = (test_asr_hidden5 + test_asr_hidden6 + test_asr_hidden7 + test_asr_hidden8) / 4.0
#     asr_feats_g3 = (test_asr_hidden9 + test_asr_hidden10 + test_asr_hidden11 + test_asr_hidden12) / 4.0
#     pred = ser_model(asr_feats_g1, asr_feats_g2, asr_feats_g3, test_hct_feats.to(torch.float))
    test_ser_loss = func(pred, torch.tensor(emolabels_test).to(DEVICE))
    for i in pred:
        predictions_test.append(i.detach().cpu().numpy())
    torch.cuda.empty_cache()
#     k += 1
        
# compute performance for each epoch
    predictions_train = np.array(predictions_train)
    predictions_test = np.array(predictions_test)
    emolabels_train = np.array(emolabels_train)
    emolabels_test = np.array(emolabels_test)
    predictions_train = [np.argmax(p) for p in predictions_train]
    acc_train = accuracy_score(emolabels_train, predictions_train)
    predictions_test = [np.argmax(p) for p in predictions_test]
    acc_test = accuracy_score(emolabels_test, predictions_test)
    
    print('Epoch:', epoch, '|train_ser_loss: %.4f' %train_ser_loss, '|train_accuracy: %.4f' %acc_train, '|test_ser_loss: %.4f' %test_ser_loss, '|test_accuracy: %.4f' %acc_test)
#     print('test: \n', confusion_matrix(emolabels_test, predictions_test))
#     print(classification_report(emolabels_test, predictions_test))

    epoch += 1
#     scheduler.step()
#     print('---testing ends---')
    
#     stop = timeit.default_timer()
#     print('Time: ', stop - start) 

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 0 |train_ser_loss: 1.3443 |train_accuracy: 0.3787 |test_ser_loss: 1.3620 |test_accuracy: 0.3951
Epoch: 1 |train_ser_loss: 1.2683 |train_accuracy: 0.4089 |test_ser_loss: 1.3047 |test_accuracy: 0.4253
Epoch: 2 |train_ser_loss: 1.1763 |train_accuracy: 0.4719 |test_ser_loss: 1.2424 |test_accuracy: 0.4922
Epoch: 3 |train_ser_loss: 1.0875 |train_accuracy: 0.5215 |test_ser_loss: 1.1850 |test_accuracy: 0.5224
Epoch: 4 |train_ser_loss: 1.0138 |train_accuracy: 0.5415 |test_ser_loss: 1.1370 |test_accuracy: 0.5453
Epoch: 5 |train_ser_loss: 0.9573 |train_accuracy: 0.5482 |test_ser_loss: 1.0975 |test_accuracy: 0.5502
Epoch: 6 |train_ser_loss: 0.9107 |train_accuracy: 0.5571 |test_ser_loss: 1.0663 |test_accuracy: 0.5649
Epoch: 7 |train_ser_loss: 0.8709 |train_accuracy: 0.5693 |test_ser_loss: 1.0404 |test_accuracy: 0.5690
Epoch: 8 |train_ser_loss: 0.8396 |train_accuracy: 0.5841 |test_ser_loss: 1.0189 |test_accuracy: 0.5682
Epoch: 9 |train_ser_loss: 0.8153 |train_accuracy: 0.5977 |test_ser_loss: 

Epoch: 79 |train_ser_loss: 0.5415 |train_accuracy: 0.7550 |test_ser_loss: 0.8738 |test_accuracy: 0.6392
Epoch: 80 |train_ser_loss: 0.5394 |train_accuracy: 0.7550 |test_ser_loss: 0.8737 |test_accuracy: 0.6392
Epoch: 81 |train_ser_loss: 0.5373 |train_accuracy: 0.7567 |test_ser_loss: 0.8736 |test_accuracy: 0.6384
Epoch: 82 |train_ser_loss: 0.5352 |train_accuracy: 0.7567 |test_ser_loss: 0.8736 |test_accuracy: 0.6384
Epoch: 83 |train_ser_loss: 0.5332 |train_accuracy: 0.7574 |test_ser_loss: 0.8736 |test_accuracy: 0.6392
Epoch: 84 |train_ser_loss: 0.5311 |train_accuracy: 0.7571 |test_ser_loss: 0.8736 |test_accuracy: 0.6392
Epoch: 85 |train_ser_loss: 0.5291 |train_accuracy: 0.7588 |test_ser_loss: 0.8735 |test_accuracy: 0.6392


In [77]:
print((0.6608+0.6670+0.6400+0.6423+0.6539)/5.0)

0.6528


In [137]:
print(torch.div(weights.detach().cpu(), torch.sum(weights.detach().cpu())))

tensor([[0.0502, 0.0684, 0.0784, 0.0937, 0.0863, 0.0936, 0.1098, 0.1087, 0.1018,
         0.0733, 0.0521, 0.0335, 0.0500]])


In [ ]:
#late fusion

torch.manual_seed(1)
ser_model1 = NeuralNet1().to(DEVICE)
ser_model2 = NeuralNet2().to(DEVICE)
params1 = ser_model1.parameters()
params2 = ser_model2.parameters()
optimizer1 = torch.optim.Adam(params1, lr=5e-4, weight_decay=5e-5)
optimizer2 = torch.optim.Adam(params2, lr=1e-4, weight_decay=1e-5)
func = nn.CrossEntropyLoss()

epochs = 100
epoch = 0
batch_size = 64

# training
while epoch < epochs:
    torch.cuda.empty_cache()
#     start = timeit.default_timer()
    print("-----epoch: ", epoch, "-----")
    predictions_train = []
    predictions1_train = []
    predictions2_train = []
    predictions_test = []
    predictions1_test = []
    predictions2_test = []

#     print('--training begins--')
    ser_model.train()
    j = 0
    while j < len(names_train)/batch_size:
        if (j+1)*batch_size > len(names_train):
            emolabels = torch.tensor(emolabels_train[j*batch_size: ]).to(DEVICE)
            indices = indices_train[j*batch_size: ]
            asr_feats0 = train_asr_hidden0[j*batch_size: ].to(DEVICE)
#             asr_feats1 = train_asr_hidden1[j*batch_size: ].to(DEVICE)
#             asr_feats2 = train_asr_hidden2[j*batch_size: ].to(DEVICE)
#             asr_feats3 = train_asr_hidden3[j*batch_size: ].to(DEVICE)
#             asr_feats4 = train_asr_hidden4[j*batch_size: ].to(DEVICE)
#             asr_feats5 = train_asr_hidden5[j*batch_size: ].to(DEVICE)
#             asr_feats6 = train_asr_hidden6[j*batch_size: ].to(DEVICE)
#             asr_feats7 = train_asr_hidden7[j*batch_size: ].to(DEVICE)
#             asr_feats8 = train_asr_hidden8[j*batch_size: ].to(DEVICE)
#             asr_feats9 = train_asr_hidden9[j*batch_size: ].to(DEVICE)
#             asr_feats10 = train_asr_hidden10[j*batch_size: ].to(DEVICE)
#             asr_feats11 = train_asr_hidden11[j*batch_size: ].to(DEVICE)
#             asr_feats12 = train_asr_hidden12[j*batch_size: ].to(DEVICE)
            hct_feats = train_hct_feats[j*batch_size: ].to(DEVICE)
            
        else:
            emolabels = torch.tensor(emolabels_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            indices = indices_train[j*batch_size:(j+1)*batch_size]
            asr_feats0 = train_asr_hidden0[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats1 = train_asr_hidden1[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats2 = train_asr_hidden2[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats3 = train_asr_hidden3[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats4 = train_asr_hidden4[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats5 = train_asr_hidden5[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats6 = train_asr_hidden6[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats7 = train_asr_hidden7[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats8 = train_asr_hidden8[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats9 = train_asr_hidden9[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats10 = train_asr_hidden10[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats11 = train_asr_hidden11[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats12 = train_asr_hidden12[j*batch_size:(j+1)*batch_size].to(DEVICE)
            hct_feats = train_hct_feats[j*batch_size:(j+1)*batch_size].to(DEVICE)
            
#         print(asr_feats_g1.shape, hct_utter.shape)
            
#         inputs = (asr_feats1 + asr_feats2 + asr_feats3 + asr_feats4 + asr_feats5 + asr_feats6 + asr_feats7 + asr_feats8 + asr_feats9 + asr_feats10 + asr_feats11 + asr_feats12) / 12.0

# ser loss
        pred1 = ser_model1(asr_feats0)
        train_ser_loss1 = func(pred1, torch.tensor(emolabels))
        for i in pred1:
            predictions1_train.append(i.detach().cpu().numpy())
        pred2 = ser_model2(hct_feats)
        train_ser_loss2 = func(pred2, torch.tensor(emolabels))
        for i in pred2:
            predictions2_train.append(i.detach().cpu().numpy())
        j += 1       

# backprop
        optimizer1.zero_grad()
        torch.nn.utils.clip_grad_norm_(params1, 1.0)
        train_ser_loss1.backward()        
        optimizer1.step()
        if epoch < 30:
            optimizer2.zero_grad()
            torch.nn.utils.clip_grad_norm_(params2, 1.0)
            train_ser_loss2.backward()        
            optimizer2.step()
        torch.cuda.empty_cache()
#     print('--training ends--')


# testing
# extract features
#     print('--testing begins--')
    ser_model.eval()

# ser loss
#     inputs_test = (test_asr_hidden1 + test_asr_hidden2 + test_asr_hidden3 + test_asr_hidden4+ test_asr_hidden5 + test_asr_hidden6 + test_asr_hidden7 + test_asr_hidden8 + test_asr_hidden9 + test_asr_hidden10 + test_asr_hidden11 + test_asr_hidden12) / 12.0
    pred1 = ser_model1(test_asr_hidden0)
    pred2 = ser_model2(test_hct_feats.to(DEVICE))
    test_ser_loss1 = func(pred1, torch.tensor(emolabels_test).to(DEVICE))
    test_ser_loss2 = func(pred2, torch.tensor(emolabels_test).to(DEVICE))
    for i in pred1:
        predictions1_test.append(i.detach().cpu().numpy())
    for i in pred2:
        predictions2_test.append(i.detach().cpu().numpy())
    torch.cuda.empty_cache()
#     k += 1
        
# compute performance for each epoch
    predictions1_train = np.array(predictions1_train)
    predictions2_train = np.array(predictions2_train)
    predictions1_test = np.array(predictions1_test)
    predictions2_test = np.array(predictions2_test)
    emolabels_train = np.array(emolabels_train)
    emolabels_test = np.array(emolabels_test)
    for i in range(len(predictions1_train)):
        if np.argmax(predictions1_train[i] == np.argmax(predictions2_train[i])):
            predictions_train.append([np.argmax(predictions1_train[i])])
        else:
            if max(predictions1_train[i]) > max(predictions2_train[i]):
                predictions_train.append([np.argmax(predictions1_train[i])])
            else:
                predictions_train.append([np.argmax(predictions2_train[i])])
#             predictions_train.append([np.argmax(predictions1_train[i] + 2*predictions2_train[i])])

    acc_train = accuracy_score(emolabels_train, predictions_train)
    for i in range(len(predictions1_test)):
        if np.argmax(predictions1_test[i] == np.argmax(predictions2_test[i])):
            predictions_test.append([np.argmax(predictions1_test[i])])
        else:
            if max(predictions1_test[i]) > max(predictions2_test[i]):
                predictions_test.append([np.argmax(predictions1_test[i])])
            else:
                predictions_test.append([np.argmax(predictions2_test[i])])
#             predictions_test.append([np.argmax(predictions1_test[i] + 2*predictions2_test[i])])
    acc_test = accuracy_score(emolabels_test, predictions_test)
 
    print('Epoch:', epoch, '|train_ser_loss: %.4f' %train_ser_loss, '|train_accuracy: %.4f' %acc_train, '|test_ser_loss1: %.4f' %test_ser_loss1, '|test_ser_loss2: %.4f' %test_ser_loss2, '|test_accuracy: %.4f' %acc_test)
#     print('test: \n', confusion_matrix(emolabels_test, predictions_test))
#     print(classification_report(emolabels_test, predictions_test))

    epoch += 1
#     scheduler.step()
#     print('---testing ends---')
    
#     stop = timeit.default_timer()
#     print('Time: ', stop - start) 

In [45]:
print(len(predictions1_test))

1150


In [ ]:
--cross-domain--

session1
fusion: 0.6586

In [ ]:
--concate--

session 1
layer 6: 0.6589
hct: 0.4856

+hct: 0.6499
+layer 0: 0.6598
+layer 1: 0.6598
+layer 2: 0.6580
+layer 3: 0.6468
+layer 4: 0.6524
+layer 5: 0.6552
+layer 6: 0.6617
+layer 7: 0.6477
+layer 8: 0.6747
+layer 9: 0.6794
+layer 10: 0.6682
+layer 11: 0.6645
+layer 12: 0.6542
    

session 2
layer 6: 0.6660
hct: 0.5132

+hct: 0.6701
+layer 0: 0.6944
+layer 1: 0.6876
+layer 2: 0.6680
+layer 3: 0.6709
+layer 4: 0.6846
+layer 5: 0.6876
+layer 6: 0.6787
+layer 7: 0.6758
+layer 8: 0.6827
+layer 9: 0.6778
+layer 10: 0.6748
+layer 11: 0.6709
+layer 12: 0.6709
    
    
session 3
layer 6: 0.6322
hct: 0.4539

+hct: 0.6426
+layer 0: 0.6609
+layer 1: 0.6530
+layer 2: 0.6530
+layer 3: 0.6496
+layer 4: 0.6391
+layer 5: 0.6452
+layer 6: 0.6617
+layer 7: 0.6626
+layer 8: 0.6678
+layer 9: 0.6643
+layer 10: 0.6609
+layer 11: 0.6470
+layer 12: 0.6574 
    

session 4
layer 6: 0.6335
hct: 0.4552

+hct: 0.6442
+layer 0: 0.6481
+layer 1: 0.6511
+layer 2: 0.6559
+layer 3: 0.6608
+layer 4: 0.6472
+layer 5: 0.6511
+layer 6: 0.6442
+layer 7: 0.6520
+layer 8: 0.6550
+layer 9: 0.6618
+layer 10: 0.6481
+layer 11: 0.6472
+layer 12: 0.6442


session 5
layer 6: 0.6278
hct: 0.4914

+hct: 0.6608
+layer 0: 0.6645
+layer 1: 0.6727
+layer 2: 0.6759
+layer 3: 0.6890
+layer 4: 0.6922
+layer 5: 0.6629
+layer 6: 0.6522
+layer 7: 0.6555
+layer 8: 0.6506
+layer 9: 0.6637
+layer 10: 0.6588
+layer 11: 0.6498
+layer 12: 0.6490 general phonetic information is captured by the last layer?

In [ ]:
--co-attention--

session 1
    
+hct: 0.6524
+layer 0: 0.6449
+layer 1: 0.6505
+layer 2: 0.6514
+layer 3: 0.6542
+layer 4: 0.6421
+layer 5: 0.6701
+layer 6: 0.6580
+layer 7: 0.6514
+layer 8: 0.6570
+layer 9: 0.6775
+layer 10: 0.6719
+layer 11: 0.6505
+layer 12: 0.6552
    
    
session 2

+hct: 0.6954
+layer 0: 0.6601
+layer 1: 0.6543
+layer 2: 0.6562
+layer 3: 0.6699
+layer 4: 0.6738
+layer 5: 0.6719
+layer 6: 0.6719
+layer 7: 0.6660
+layer 8: 0.6925
+layer 9: 0.6670
+layer 10: 0.6748
+layer 11: 0.6621
+layer 12: 0.6601
    
    
session 3

+hct: 0.6278
+layer 0: 0.6365
+layer 1: 0.6374
+layer 2: 0.6557
+layer 3: 0.6487
+layer 4: 0.6661
+layer 5: 0.6591
+layer 6: 0.6513
+layer 7: 0.6591
+layer 8: 0.6626
+layer 9: 0.6696
+layer 10: 0.6470
+layer 11: 0.6435
+layer 12: 0.6339
    
    
session 4

+hct: 0.6355
+layer 0: 0.6365
+layer 1: 0.6462
+layer 2: 0.6520
+layer 3: 0.6637
+layer 4: 0.6647
+layer 5: 0.6326
+layer 6: 0.6316
+layer 7: 0.6540
+layer 8: 0.6452
+layer 9: 0.6365
+layer 10: 0.6345
+layer 11: 0.6384
+layer 12: 0.6238


session 5

+hct: 0.6400
+layer 0: 0.6531
+layer 1: 0.6661
+layer 2: 0.6571
+layer 3: 0.6784
+layer 4: 0.6857
+layer 5: 0.6531
+layer 6: 0.6506
+layer 7: 0.6539
+layer 8: 0.6596
+layer 9: 0.6506
+layer 10: 0.6441
+layer 11: 0.6384
+layer 12: 0.6335

In [ ]:
--average--

session 1: 0.6365
session 2: 0.6533
session 3: 0.6183
session 4: 0.6530
session 5: 0.6163